# 5. Geospatial python

Because of the wide array (pun intended) of open source data science modules that are accessible with python, it has emerged as a flagship language for processing, analyzing, and visualizing geospatial data. There are several key geospatial packages available in python: 

* [Geopandas](https://geopandas.org/en/stable/) - processing vector data (e.g. shapfiles, `.GeoJSON`, `.KML`, see [here](https://gdal.org/en/stable/drivers/vector/index.html) for a full list of geospatial vector filetypes)
* [Rasterio](https://rasterio.readthedocs.io/en/stable/) - processing raster data (e.g. satellite imagery in `.tif` or `.tiff` format)
* [Xarray](https://docs.xarray.dev/) - multidimensional array analysis (e.g. time series gridded or satellite datasets in `.nc` or `.hdf` format)
* [GDAL](https://gdal.org/) - the key library (originally written in C and C++) that underscores most of the raster and vector processing operations in the above libraries, as well as GUI-based GIS software (ArcMap, QGIS, Google Earth/Maps, etc)

## Geopandas

Remember `pandas` from last module? Enter `geopandas` - a natural geospatial extension to store tabular vector data (think points, lines, and polygons) and simplify operations by keeping with pandas-like syntax. 

Let's load the same `stations.csv` file and convert it from a `pandas.DataFrame` to a `geopandas.GeoDataFrame`:

In [ ]:
import pandas as pd

In [ ]:
url = "https://raw.githubusercontent.com/py4wrds/py4wrds/refs/heads/module-4/data/gwl/stations.csv"
df  = pd.read_csv(url)

In [ ]:
df.head()

In [ ]:
# Create a GeoDataFrame from the DataFrame

import geopandas as gp

stn_gdf = gp.GeoDataFrame(df, 
    geometry=gp.points_from_xy(df['LONGITUDE'], df['LATITUDE']),
    crs="EPSG:4326"  # Set the coordinate reference system (CRS)
)

In [ ]:
# Plot our results:
stn_gdf.plot()

When printed, the tabular data is identical, and all `pandas`-like syntax can be used to sort and filter data:

In [ ]:
# Get unique county names
print(stn_gdf['COUNTY_NAME'].unique())
# Plotting all stations in SLO county only 
stn_gdf[stn_gdf['COUNTY_NAME'] == 'San Luis Obispo'].plot()

### Reading and writing files

Geopandas excels at making it easy to read and write vector files of several formats, which facilitates conversions and interoperability. 

simply `import geopandas as gp` and then read any common vector file format with the `gp.read_file('filename')` function, which works for common formats such as `.shp` (and supporting `.dbf` and `.shx` files), `kml`, `kmz`, `geojson`, etc..

In [ ]:
# Read a geojson file
url = 'https://raw.githubusercontent.com/scottpham/california-counties/refs/heads/master/caCountiesTopo.json'
gdf = gp.read_file(url)
gdf.head()

In [ ]:
gdf.plot()

To write files, simply use the syntax: `gdf.to_file("path/to/out_file.geojson")` syntax. We recommend storing files as `.geojson` due to readability and simplicity to store and retrieve. 

In [ ]:
gdf.to_file("ca_counties.geojson")

We received a warning that our CRS was not defined. We can define it using `gdf.set_crs('epsg:4236')`. Similar to other `DataFrame` and `pandas`-like operations, we must specify the `inplace = True` argument: 

In [ ]:
gdf.set_crs('epsg:4326', inplace = True)
gdf.to_file("ca_counties.geojson")

In [ ]:
# verify that file was written in our current directory: 
import os
[x for x in os.listdir(os.getcwd()) if 'geojson' in x]

### Managing Projections

We can inspect the [coordinate system](https://en.wikipedia.org/wiki/Spatial_reference_system) by using the `gdf.crs` syntax: 

In [ ]:
# Geographic CRS example:
stn_gdf.crs

#### Reprojections

Let's reproject the geodataframe into a projected CRS for California - UTM Zone 10 N, which is `epsg:32611`: 

In [ ]:
gdf_rpj = gdf.to_crs('epsg:32611')
gdf_rpj.crs

the `estimate_utm_crs()` function may be convenient for going between geographic and projected CRS: df.estimate_utm_crs() 

In [ ]:
gdf.estimate_utm_crs() 

### Geometric Operations

We can easily calculate geometry properties such as area, centroids, bounds, and distances using geopandas.

#### Calculating Areas

In [ ]:
# Calculate area for each polygon in a UTM CRS
# Note - the units depend on the coordinate system, 
# Be sure to reproject appropriately and perform conversions

gdf_rpj['area_sqkm'] = gdf_rpj.area * 1e-6 # Sq m to sq km

In [ ]:
gdf_rpj.head()

#### Centroids

Calculating centroids can be done with the `gdf.centroid` function:

In [ ]:
print(gdf_rpj.centroid.head())
gdf_rpj.centroid.plot()

#### Boundaries

Similarly, boundaries can be obtained with the `gdf.boundary` function: 

In [ ]:
# Get the boundary of each polygon
print(gdf.boundary.head())
gdf.boundary.plot()

#### Distances

Similarly, distances between two points can be computed with the `distance` function: 

Let's calculate the distance between each of the stations in our dataset

In [ ]:
stn_gdf.plot()

In [ ]:
# Reproject to UTM zone 11N
stn_gdf.to_crs(stn_gdf.estimate_utm_crs(), inplace = True)

# calculate distance between each station from the first station
stn_gdf['dist_from_stn_1'] = stn_gdf.distance(stn_gdf.iloc[0].geometry)

In [ ]:
# Plot stations, colored by distance 
ax = stn_gdf.plot(column = 'dist_from_stn_1', legend = True)
# Plot the first station 
stn_gdf.loc[[0],'geometry'].plot(markersize = 50,color = 'red', ax = ax, label = '0th station')
# add a legend a title 
ax.legend()
ax.set_title('dist from 0th station')

### Plotting 

Just above we see an example of how to color a `GeoDataFrame` by a column, by specifying the `column = 'column_name'` argument into `gdf.plot()`. 

We can create interactive plots with the `gdf.explore()` funtion: 

In [ ]:
slo_gdf = stn_gdf[stn_gdf['COUNTY_NAME'] == 'San Luis Obispo']

slo_gdf.explore(legend=False)

### Spatial Operations 

Geopandas makes it easy to perform basic spatial operations such as Buffers, Convex Hull, and Spatial Joins

#### Buffer

In [ ]:
# Apply a 500m buffer to the first 5 points in the SLO well data 
ax = slo_gdf[:5].buffer(1000).plot()
# Plot the same 5 but with a smaller (50m) buffer 
slo_gdf[:5].buffer(100).plot(ax = ax, color = 'red')

#### Convex Hull

In [ ]:
gdf["convex_hull"] = gdf.convex_hull
# Plot the convex hulls
ax = gdf["convex_hull"].plot(alpha=0.5, color="lightblue", edgecolor="black")
ax.set_title("Convex Hull of CA Counties")

#### Spatial Union

In [ ]:
slo_gdf['buf_geom'] = slo_gdf[:5].buffer(1000)
slo_gdf['buf_geom'].plot()

In [ ]:
slo_gdf['buf_geom'].head()

In [ ]:
slo_gdf['buf_geom'].union_all()

### Spatial Joins, Queries, and Relations

#### Intersections
To check for spatial intersection between two datasets, we can use the `intersection` function: 

In [ ]:
# Check which buffered boroughs intersect with SLO's geometry
slo_co = gdf[gdf['name'] == 'San Luis Obispo'].to_crs(gdf.estimate_utm_crs())

slo_co.plot()

In [ ]:
slo_co_stns = gp.overlay(stn_gdf, slo_co, how='intersection')

In [ ]:
slo_co_stns.plot()

In [ ]:
ax = slo_co.plot(color = 'white', edgecolor = 'red')
slo_co_stns.plot(alpha = 0.5, markersize = 5, color = 'gray', label = 'stations', ax = ax)

## Rasterio

Rasterio is a powerful library for working with [raster data](https://en.wikipedia.org/wiki/Raster_graphics) (think: images or grids of numbers with pixels) written on top of the Geospatial Data Abstraction Library [(GDAL)](https://gdal.org/), making it easier to perform a wide variety of raster processing and visualization tasks. 

### Reading and inspecting data 

We can open geospatial raster datasets (e.g. `.tif` and `.tiff` files) using the `.open` function:

In [ ]:
import rasterio as rio

raster_path = "https://github.com/opengeos/datasets/releases/download/raster/dem_90m.tif"
src = rio.open(raster_path)
print(src)

Notice that we are reading a file from a URL here, but this function works identically for local files. 

Once a dataset has been opened, we can inspect lots of useful information, including:
* Projection information
* Bounding box
* Spatial resolution
* Dimensions - width / height
* Other metadata

In [ ]:
# Inspect crs
src.crs

In [ ]:
# Inspect metadata
src.meta

In [ ]:
# Inspect spatial res
src.res

In [ ]:
# Inspect dimensions or width / height 
print(src.shape, src.width, src.height)

In [ ]:
# Inspect bounding box
src.bounds

### Plotting and visualizing

We can use the `read` function to read raster bands from `.tiff` or `.tif` files as `numpy` arrays, and then make use the `imshow` function from the trusty module `matplotlib`! 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

arr = src.read(1)
plt.imshow(arr); plt.colorbar()

In the above code, we are reading the first band from the raster using `src.read(1)`. For multiband rasters, we can read the other bands using the same syntax. 

Note that **this method does not preserve any projection information in the plot** - the x and y axes are just the number of rows / cols in the dataset. 

If we want to preserve any geospatial coordinate information, we can use the rasterio plotting module:

In [ ]:
import rasterio.plot 

rio.plot.show(src)

#### Formatting Plots

We can leverage [matplotlib's functionality](https://matplotlib.org/stable/gallery/images_contours_and_fields/image_demo.html) to style our plots, including adding titles and choosing colormaps:

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))
rasterio.plot.show(src, cmap="terrain", ax=ax, title="Digital Elevation Model (DEM)")
plt.show()

#### Plotting vector and raster data

In [ ]:
Tuolumne_co = gdf[gdf['name'] == 'Tuolumne']
Tuolumne_co.to_crs(src.crs, inplace = True)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))
rasterio.plot.show(src, cmap="terrain", ax=ax, title="Digital Elevation Model (DEM) and Tuolumne co")
Tuolumne_co.plot(ax=ax, edgecolor="red", facecolor="none", lw=2)

### Clipping raster to vector data

A common task involves clipping a raster to a vector dataset. We can use `rasterio.mask` to clip the DEM above to the boundary of Tuolumne county, defined in the `ca_counties.geojson` file: 

In [ ]:
import rasterio.mask 

# Read raster 
raster_path = "https://github.com/opengeos/datasets/releases/download/raster/dem_90m.tif"
src = rio.open(raster_path)

# Read shape, isolate Tuolumne
shp_geom = gp.read_file("ca_counties.geojson")
Tuolumne_co = shp_geom[shp_geom['name'] == 'Tuolumne']

# Ensure CRS are consistent
Tuolumne_co.to_crs(src.crs, inplace = True)

# Perform clip 
out_image, out_transform = rio.mask.mask(src, Tuolumne_co['geometry'], crop=True)

In [ ]:
# Verify results:

rasterio.plot.show(out_image[0,:,:]);

Notice that rasterio returns by default an array with 3 dimensions (`out_image`), for which we have selected the first band manually. This functionality exists to deal with multiband rasters. 

### Writing rasters

Let's save the raster we just clipped

In [ ]:
# Set the output file 
output_raster_path = "./data/clipped.tif"

# Extract some information from the image we processed 
kwargs = src.meta
kwargs.update(
    height=out_image.shape[1],
    width=out_image.shape[2],
    count=out_image.shape[0],
    dtype=out_image.dtype,
    crs=src.crs)

# Save
with rasterio.open(output_raster_path, "w", **kwargs) as dst:
    dst.write(out_image)

print(f"Raster data has been written to {output_raster_path}")


### Reprojecting Rasters

We use the `rasterio.warp` module to change raster projections. 

Let's reproject our clipped raster to the WGS 84 (EPSG:3857) CRS and save the reprojected raster to a new file.

In [ ]:
from rasterio.warp import calculate_default_transform, reproject, Resampling

# Set output filename 
raster_path = "./data/clipped.tif"
dst_crs = "EPSG:4326"  # WGS 84
output_reprojected_path = "./data/clipped_reprojected.tif"

with rasterio.open(raster_path) as src:
    transform, width, height = calculate_default_transform(
        src.crs, dst_crs, src.width, src.height, *src.bounds)

    profile = src.profile
    profile.update(crs=dst_crs, transform=transform, width=width, height=height)

    with rasterio.open(output_reprojected_path, "w", **profile) as dst:
        for i in range(1, src.count + 1):
            reproject(
                source=rasterio.band(src, i),
                destination=rasterio.band(dst, i),
                src_transform=src.transform,
                src_crs=src.crs,
                dst_transform=transform,
                dst_crs=dst_crs,
                resampling=Resampling.nearest,
            )
print(f"Reprojected raster saved at {output_reprojected_path}")


### Multiband Rasters 

Satellite data is generally composed of a number of [spectral bands](https://landsat.gsfc.nasa.gov/satellites/landsat-8/landsat-8-bands/). Each band measures the light reflected between two wavelengths (hence the term 'band'). Rasterio allows us to work with these bands individually and in concert. 

For example, NASA's Landsat Satellites record the following bands: 

| Name   | Wavelength       | Description                                        |
|--------|------------------|----------------------------------------------------|
| SR_B1  | 0.435-0.451 μm   | Band 1 (ultra blue, coastal aerosol) surface reflectance |
| SR_B2  | 0.452-0.512 μm   | Band 2 (blue) surface reflectance                   |
| SR_B3  | 0.533-0.590 μm   | Band 3 (green) surface reflectance                  |
| SR_B4  | 0.636-0.673 μm   | Band 4 (red) surface reflectance                    |
| SR_B5  | 0.851-0.879 μm   | Band 5 (near infrared) surface reflectance          |
| SR_B6  | 1.566-1.651 μm   | Band 6 (shortwave infrared 1) surface reflectance   |
| SR_B7  | 2.107-2.294 μm   | Band 7 (shortwave infrared 2) surface reflectance   |

Let's plot all the bands in this raster

In [ ]:
# Read multiband Landsat image: 
raster_path = "https://github.com/opengeos/datasets/releases/download/raster/LC09_039035_20240708_90m.tif"
src = rasterio.open(raster_path)
print(src)

# Specify band names: 
band_names = ["Coastal Aerosol", "Blue", "Green", "Red", "NIR", "SWIR1", "SWIR2"]

In [ ]:
# Plot raster: 

fig, axes = plt.subplots(nrows=3, ncols=2, figsize=(8, 10))
axes = axes.flatten()  # Flatten the 2D array of axes to 1D for easy iteration

for band in range(1, src.count):
    data = src.read(band)
    ax = axes[band - 1]
    im = ax.imshow(data, cmap="gray", vmin=0, vmax=0.5)
    ax.set_title(f"Band {band_names[band - 1]}")
    ax.axis("off")
    fig.colorbar(im, ax=ax, label="Reflectance", shrink=0.5)

plt.tight_layout()
plt.show()


### True color image

In order to visualize a true color image, we need to visualize band 2 (blue), band 3 (green), and band 4 (red) in the same image: 

In [ ]:
nir_band = src.read(5)
red_band = src.read(4)
green_band = src.read(3)

# Stack the bands into a single array
rgb = np.dstack((nir_band, red_band, green_band)).clip(0, 1)

# Plot the stacked array
plt.figure(figsize=(6, 6))
plt.imshow(rgb)
plt.axis("off")
plt.title("Bands NIR, Red, and Green combined")
plt.show()

#### Band math: Calculating NDVI 

A common metric that we can derive from multiple bands is [Normalized Difference Vegetation Index (NDVI)](https://en.wikipedia.org/wiki/Normalized_difference_vegetation_index), which is defined as: 

```
$$NDVI = (NIR - Red) / (NIR + Red)$$
```

In [ ]:
# NDVI Calculation: NDVI = (NIR - Red) / (NIR + Red)
ndvi = (nir_band - red_band) / (nir_band + red_band)
ndvi = ndvi.clip(-1, 1)

plt.figure(figsize=(6, 6))
plt.imshow(ndvi, cmap="RdYlGn", vmin=-1, vmax=1)
plt.colorbar(label="NDVI", shrink=0.5)
plt.title("NDVI")
plt.xlabel("Column #")
plt.ylabel("Row #")
plt.show()

## Xarray

Xarray makes working with labelled multi-dimensional arrays in Python simple, efficient, and fun!

Xarray extends NumPy functionality by providing data structures specifically for geospatial multi-dimensional arrays:

    * DataArray: A labeled, multi-dimensional array, which includes dimensions, coordinates, and attributes.

    * Dataset: A collection of DataArray objects that share the same dimensions.


### Xarray data representation
Xarray represents stacks of images as 'cubes', where two axes represent the latitude and longitude coordinates of spatial data, and a third axis represents time. Note that an xarray dataset can contain more than one variable! 

The schematic below shows examples for temperature and precipitation

![xarray](img/5-xarray-dataset-diagram.png)

### Loading Data and inspecting attributes

This data was downloaded from NOAA: https://psl.noaa.gov/data/gridded/data.ncep.reanalysis.html 

In [ ]:
import xarray as xr 

ds = xr.open_dataset("./data/runof.sfc.mon.mean.nc")
ds

In [ ]:
ds["runof"].mean(dim="time").plot()

In [ ]:
# Select the data array describing runoff
ro = ds["runof"]

In [ ]:
# Plot some of the data array attributes

# Obtain a numpy array of the values
print(ro.values.shape)
print(ro.values)

In [ ]:
print(ro.coords)

In [ ]:
print(ro.attrs)

In [ ]:
print(ro.dims)

### Filtering and subsetting

We can easily select data based on dimension labels, which is very intuitive when working with geospatial data.

In [ ]:
# Select data for a specific time and location
selected_data = ro.sel(time="2000-01-01", lat=40.0, lon=120.0, method = 'nearest')
selected_data

In [ ]:
# Slice data across a range of times
time_slice = ro.sel(time=slice("2013-01-01", "2013-01-31"))
time_slice

In [ ]:
# Calculate the mean SWE over time
mean_ro = ro.mean(dim="time")
mean_ro

### Plotting and visualization

Xarray provides builtin plotting methods that can be readily modified and integrated with standard `matplotlib` syntax

In [ ]:
mean_ro.plot(cmap="jet", figsize=(10, 6))
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.title("Mean Monthly Runoff")

We can use the `.isel` method to select a spatial subset of the data by index:

In [ ]:
cropped_ds = mean_ro.isel(lat=slice(20,40), lon=slice(120,160))
cropped_ds.plot()


In [ ]:
# Plot a time series for a specific location
ro.sel(lat=35.0, lon=240, method = 'nearest').plot()
plt.show()